In [1]:
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
import nest_asyncio

nest_asyncio.apply()


In [4]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [5]:
os.makedirs('data/paul_graham/', exist_ok=True)
!curl -o "data/paul_graham/paul_graham_essay.txt" "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 75042  100 75042    0     0   232k      0 --:--:-- --:--:-- --:--:--  234k


In [7]:
# load data
from llama_index.core import SimpleDirectoryReader

reader  = SimpleDirectoryReader(input_dir="./data/paul_graham/")


In [8]:
documents= reader.load_data()

In [9]:
from llama_index.core import VectorStoreIndex

pg_vector_index=VectorStoreIndex.from_documents(documents=documents,
    use_async=True,
    show_progress=True,
)


c:\Code\Github\LlamaIndex\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Generating embeddings: 100%|██████████| 1/1 [00:03<00:00,  3.14s/it]


# Persist Locally

In [10]:
pg_vector_index.storage_context.persist(persist_dir="./pg_vector_index")

# Load the Index (from storage)

 When you need to use the index again, instead of re-indexing, you can load it from the persisted storage using load_index_from_storage.

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

# Rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="pg_vector_index")

# Load the index
pg_vector_index = load_index_from_storage(storage_context)

In [13]:
from llama_index.llms.openai import OpenAI

gpt_4o_mini = OpenAI(temperature=0, model="gpt-4o-mini")

chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo")

In [14]:
from llama_index.core.composability import QASummaryQueryEngineBuilder


query_engine_builder = QASummaryQueryEngineBuilder(
    llm=gpt_4o_mini,
)

In [21]:
query_engine = query_engine_builder.build_from_documents(documents)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [22]:
response = query_engine.query(
    "Can you give me a summary of the author's life?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 1: The question asks for a summary, which aligns with the purpose of the second index..
Selecting query engine 1: The question asks for a summary, which aligns with the purpose of the second index..
Selecting query engine 1: The question asks for a summary, which aligns with the purpose of the second index..
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [24]:
print(response)

The author began his journey with writing and programming at a young age, initially focusing on short stories and early programming on an IBM 1401. His interest in computers grew with the advent of microcomputers, leading him to write simple games and a word processor on a TRS-80. Although he initially planned to study philosophy in college, he found it unfulfilling and switched to artificial intelligence (AI), inspired by literature and documentaries.

After completing his undergraduate studies, he pursued a PhD in computer science, where he became disillusioned with the state of AI at the time. He then shifted his focus to Lisp programming and began writing a book on the subject. His career took a turn when he co-founded Viaweb, an early e-commerce platform, which was later acquired by Yahoo. This experience provided him with financial security, allowing him to explore painting, a long-held passion.

Following a period of painting and personal exploration, he transitioned into ventur

In [25]:
response = query_engine.query(
    "What did the author do growing up?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.query_engine.router_query_engine:Selecting query engine 0: The question likely requires retrieval of specific context from documents related to the author's upbringing..
Selecting query engine 0: The question likely requires retrieval of specific context from documents related to the author's upbringing..
Selecting query engine 0: The question likely requires retrieval of specific context from documents related to the author's upbringing..
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.c